In [62]:
import pandas as pd
# читаем содержимое фалов
bulls = pd.read_csv('bulls.csv')  # быки-кандидаты на осеменение
cows = pd.read_csv('cows.csv')  # коровы, которых нужно осеменить
date_map = {'id': 'str', 'mother_id': 'str', 'father_id': 'str'}
pedigree = pd.read_csv('pedigree.csv', dtype=date_map)  # родословные животных

def clear_data(df):
    """ Заполняем пропущенные ebv средним значением и сортируем по ebv (в приоритете те, с кем среднее будет больше) """
    return df.fillna(bulls.mean(numeric_only=True)).sort_values('ebv', ascending=False)

bulls = clear_data(bulls)
cows = clear_data(cows)

In [63]:
import numpy as np
""" Каждый бык может осеменить только 10% коров """
max_cows_for_bull = int(np.floor(0.1 * len(cows)))
bulls['free_for_match'] = max_cows_for_bull  # счетчик, сколько коров может оплодотворить
print('Один бык может осеменить {max_cows} коров'.format(max_cows=max_cows_for_bull))

Один бык может осеменить 1717 коров


In [64]:
import networkx as nx
""" Приводим список родителей к более удобному виду - графу родства """
pedigree_graph = nx.DiGraph()
nodes = set(pedigree['id']).union(set(pedigree['mother_id']), set(pedigree['father_id']))
pedigree_graph.add_nodes_from(nodes)
# pedigree_graph.nodes()


In [65]:
def common_ancestors(pedigree_graph, bull_id, cow_id):
    """ Находим все узлы (родственников) от переданого id животного по графу родства и ищем общих предков """
    a1 = nx.ancestors(pedigree_graph, bull_id)
    a2 = nx.ancestors(pedigree_graph, cow_id)
    return a1 & a2

def is_suitable_pair(pedigree_graph, bull_id, cow_id, max_depth=3):
    """ 5% родства - это троюродные и дальше, то есть глубина больше или равна 3 """
    def get_ancestors_with_depth(node):
        visited = {}
        queue = [(node, 0)]
        while queue:
            current, depth = queue.pop(0)
            if current in visited and visited[current] <= depth:
                continue
            visited[current] = depth
            if depth < max_depth:
                for parent in pedigree_graph.predecessors(current):
                    # Идем по предыдущим узлам от переданного, чтобы найти родителей нужной глубины
                    queue.append((parent, depth + 1))
        return visited

    a1 = get_ancestors_with_depth(bull_id)
    a2 = get_ancestors_with_depth(cow_id)
    common = set(a1.keys()) & set(a2.keys())
    # Фильтруем только тех предков, которые находятся ближе max_depth
    for ancestor in common:
        if a1[ancestor] < max_depth and a2[ancestor] < max_depth:
            return True
    return False

In [66]:
""" Добавила флаг - отмечаем коров, которым уже подобрана оптимальная пара """
print(cows.count())
cows['used'] = False
cows.head()

id     17177
ebv    17177
dtype: int64


,id,ebv,used
4838,DE00000028635,2046.2,False
6810,FR00000011732,1981.1,False
9716,DE00000023145,1840.5,False
8011,DE00000014083,1814.8,False
5699,NL00000004886,1788.7,False


In [67]:
valid_pairs = []
for bull_idx, bull in bulls.iterrows():
    available_cows = cows[~cows['used']]  # доступные коровы
    for cows_idx, cow in available_cows.iterrows():
        if not is_suitable_pair(pedigree_graph, bull['id'], cow['id'], max_depth=3):
            valid_pairs.append((cow['id'], bull['id']))
            cows.loc[cows_idx, 'used'] = True  # этих коров уже распределили по парам и больше не используем
            bulls.loc[bull_idx, 'free_for_match'] -= 1  # уменьшаем счетчик
            # print(bull['id'], cow['id'])
        if bulls.loc[bull_idx, 'free_for_match'] == 0:
            break  # Переходим к следубщему быку
valid_pairs

US00000000795 DE00000028635
US00000000795 FR00000011732
US00000000795 DE00000023145
US00000000795 DE00000014083
US00000000795 NL00000004886
US00000000795 DE00000027440
US00000000795 US00000069140
US00000000795 GB00000083297
US00000000795 GB00000024686
US00000000795 GB00000017595
US00000000795 NL00000027867
US00000000795 DE00000023455
US00000000795 GB00000009133
US00000000795 RU00000045502
US00000000795 US00000011575
US00000000795 US00000033490
US00000000795 NL00000028599
US00000000795 US00000006228
US00000000795 NL00000074190
US00000000795 DE00000021659
US00000000795 RU00000029036
US00000000795 NL00000017666
US00000000795 GB00000029511
US00000000795 NL00000023844
US00000000795 US00000014607
US00000000795 DE00000026925
US00000000795 NL00000016724
US00000000795 NL00000009484
US00000000795 DE00000022602
US00000000795 NL00000020384
US00000000795 RU00000008446
US00000000795 DE00000015455
US00000000795 NL00000036352
US00000000795 US00000040481
US00000000795 RU00000027070
US00000000795 NL0000

[('DE00000028635', 'US00000000795'),
 ('FR00000011732', 'US00000000795'),
 ('DE00000023145', 'US00000000795'),
 ('DE00000014083', 'US00000000795'),
 ('NL00000004886', 'US00000000795'),
 ('DE00000027440', 'US00000000795'),
 ('US00000069140', 'US00000000795'),
 ('GB00000083297', 'US00000000795'),
 ('GB00000024686', 'US00000000795'),
 ('GB00000017595', 'US00000000795'),
 ('NL00000027867', 'US00000000795'),
 ('DE00000023455', 'US00000000795'),
 ('GB00000009133', 'US00000000795'),
 ('RU00000045502', 'US00000000795'),
 ('US00000011575', 'US00000000795'),
 ('US00000033490', 'US00000000795'),
 ('NL00000028599', 'US00000000795'),
 ('US00000006228', 'US00000000795'),
 ('NL00000074190', 'US00000000795'),
 ('DE00000021659', 'US00000000795'),
 ('RU00000029036', 'US00000000795'),
 ('NL00000017666', 'US00000000795'),
 ('GB00000029511', 'US00000000795'),
 ('NL00000023844', 'US00000000795'),
 ('US00000014607', 'US00000000795'),
 ('DE00000026925', 'US00000000795'),
 ('NL00000016724', 'US00000000795'),
 

In [69]:
valid_df = pd.DataFrame(valid_pairs, columns=['cow_id', 'bull_id'])
valid_df.to_csv("cow_bull_assignments.csv", index=False)